In [1]:
%run raceTimePredictor.py

Load Data
Start training

Iteration  1
train size:  69  test size:  4
dist: 8.01 km, elev: 199.81 m, time: 31.75 min, predicted time: 28.05 min, accuracy: 88.34 %
dist: 21.09 km, elev: 83.72 m, time: 81.98 min, predicted time: 82.47 min, accuracy: 99.41 %
dist: 9.88 km, elev: 16.74 m, time: 37.03 min, predicted time: 33.67 min, accuracy: 90.91 %
dist: 9.97 km, elev: 5.81 m, time: 36.52 min, predicted time: 33.91 min, accuracy: 92.85 %
Test set evaluation
Mean Squared Error: 8.0211 min
Mean Absolute Error: 2.5404 min
Root Mean Squared Error: 2.8321 min
Accuracy: 92.88 % 

Iteration  2
train size:  69  test size:  4
dist: 9.52 km, elev: 125.39 m, time: 37.28 min, predicted time: 35.17 min, accuracy: 94.32 %
dist: 10.07 km, elev: 62.84 m, time: 37.97 min, predicted time: 37.50 min, accuracy: 98.77 %
dist: 5.37 km, elev: 69.45 m, time: 19.58 min, predicted time: 19.98 min, accuracy: 97.99 %
dist: 5.09 km, elev: 38.43 m, time: 18.45 min, predicted time: 17.91 min, accuracy: 97.08 %
Test set

In [1]:
%run raceTimePredictor.py

Load Data
Start training

Iteration  1
train size:  167  test size:  4
dist: 8.01 km, elev: 199.81 m, time: 31.75 min, predicted time: 30.61 min, accuracy: 96.41 %
dist: 21.09 km, elev: 83.72 m, time: 81.98 min, predicted time: 81.81 min, accuracy: 99.79 %
dist: 9.88 km, elev: 16.74 m, time: 37.03 min, predicted time: 35.96 min, accuracy: 97.11 %
dist: 9.97 km, elev: 5.81 m, time: 36.52 min, predicted time: 36.99 min, accuracy: 98.71 %
Test set evaluation
Mean Squared Error: 0.6739 min
Mean Absolute Error: 0.7138 min
Root Mean Squared Error: 0.8209 min
Accuracy: 98.00 % 

Iteration  2
train size:  167  test size:  4
dist: 9.52 km, elev: 125.39 m, time: 37.28 min, predicted time: 34.62 min, accuracy: 92.84 %
dist: 10.07 km, elev: 62.84 m, time: 37.97 min, predicted time: 36.79 min, accuracy: 96.90 %
dist: 5.37 km, elev: 69.45 m, time: 19.58 min, predicted time: 15.66 min, accuracy: 79.98 %
dist: 5.09 km, elev: 38.43 m, time: 18.45 min, predicted time: 12.82 min, accuracy: 69.50 %
Test s